In [1]:
from gap_statistic import OptimalK

from sklearn.datasets.samples_generator import make_blobs



In [16]:
%%time
X, y = make_blobs(n_samples=int(1e6), n_features=5, centers=3, random_state=100)

print('Data shape: ', X.shape)

ok = OptimalK()
gap, df = ok(X)

print(gap)

Data shape:  (1000000, 5)
3
CPU times: user 1.58 s, sys: 72 ms, total: 1.66 s
Wall time: 1min 15s


In [333]:

import numpy as np
from scipy.cluster.vq import kmeans2




centeroids, labels = kmeans2(X, 3, iter=10, minit='points')
disp = np.sum(np.sum([np.abs(inst - centeroids[label])**2 for inst, label in zip(X, labels)]))
disp

20060.398035053498

TypeError: return arrays must be of ArrayType

In [326]:
%timeit np.subtract(2, 4)

1 µs ± 12 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [310]:
%%timeit
from sklearn.cluster import KMeans

km = KMeans(3, )
km.fit(X)
km.inertia_

40.8 ms ± 2.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
